In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import groupyr as gpr
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LassoCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
# Open the files and convert to dataframes
df_train = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv', sep = ",", header = 0, index_col = 0)
df_test = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv', sep = ",", header = 0, index_col = 0)

# Explore dataset
df_train.head()

In [7]:
df_train.describe()


Using information from: https://towardsdatascience.com/7-ways-to-handle-missing-values-in-machine-learning-1a6326adf79e
<br>Handling of Null Values:
<br>1) if columns has 50% misisng values, drop them
<br>2) Impute missing values with mean/median or 0, depending on the data in column.
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - or can use interpolation for time series dataset, Last observation carried forward (LOCF) method for longitudinal behaviours
<br>3) For categorical data, if missing values are minimal, replace wiht most frequent, if not replace with a new cateogry variable
<br>4) Use of ML algorithms: KNN or navies Bayes (BUT scikit does not support), or Random Forest
<br>5) Regression or Classification models (uses correlation advantage as compared to (4))
<br>6) Deep Learning Library (Datawig)using deep neural networks to predict missing values
<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; - need to learn more on this

In [8]:
bar = df_train.isna().sum().reset_index().sort_values(0, axis = 0)
bar = bar.drop(bar.index[(bar.iloc[:,1] == 0)], axis=0)

fig, ax = plt.subplots()

y_pos = np.arange(len(bar.iloc[:,0]))

ax.barh(y_pos, bar.iloc[:,1], align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(bar.iloc[:,0])
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('NA values')
ax.set_title('Columns with NA values')

plt.show()

In [9]:
over50_null = (df_train.isna().sum()/1460)[df_train.isna().sum()/1460 > 0.5].index.values.tolist()
df_train.drop(columns=(over50_null), inplace = True)
df_test.drop(columns=(over50_null), inplace = True)

df_train


In [10]:
# Check for the rest of null values
df_train.isna().any()[lambda x: x].index.values.tolist()


In [11]:
df_test.isna().any()[lambda x: x].index.values.tolist()

In [12]:
# Separate columns to numerical and categorical for imputation of missing values
numerical = df_train._get_numeric_data().columns
categorical = list(set(df_train.columns) - set(numerical))

In [13]:
# numerical without the SalePrice (answer)
numerical = numerical.drop("SalePrice")
numerical

In [14]:
# categorical variables
categorical

In [15]:
# Fill up all na categorical missing values with None as it indicates that there aren't any measurements to get
def fill_data_cat(data):
    
    for col in categorical:
        data.loc[data[col].isna(), col] = 'None'
    
    return data

In [16]:
# Categorical column missing values filled
df_train_CatFilled = fill_data_cat(df_train)
df_test_CatFilled = fill_data_cat(df_test)

# Checking if all categorical columns filled
print(df_train_CatFilled.isna().any()[lambda x: x])
print(df_test_CatFilled.isna().any()[lambda x: x])

In [17]:
# numerical values to be 0 as nothing to measure (time data might have issues)
def fill_data_num(data):
    for col in numerical:
        data.loc[data[col].isna(), col] = '0'
    return data


In [18]:
# Numerical column missing values filled
df_train_filled = fill_data_num(df_train_CatFilled)
df_test_filled = fill_data_num(df_test_CatFilled)

# Checking if all columns filled now
print(df_train_filled.isna().any()[lambda x: x])
print(df_test_filled.isna().any()[lambda x: x])

In [19]:
# Now we have settled all missing values
# There are still lots of dimensions (columns) so we will try to reduce 

# There is only one value 'NoSeWa' in one row while the rest is 'AllPub'.
# It is not enough to be considered useful to be kept just for one row difference. 
# Remove row as well as 
df_train_filled["Utilities"].value_counts()

In [20]:
df_train_filled.drop(df_train_filled.index[(df_train_filled['Utilities'] == 'NoSeWa')], axis=0, inplace = True)
df_train_filled.drop('Utilities', axis=1, inplace = True)
df_test_filled.drop('Utilities', axis=1, inplace = True)
df_train_filled

In [21]:
# We see a lot of similarities like that amongst some columns
df_train_filled.columns

In [22]:
# We don't need TotalBsmtSF as we have the total separated amongst BsmtFinSF1 and BsmtFinSF2
df_train_filled.drop('TotalBsmtSF', axis = 1, inplace = True)
df_test_filled.drop('TotalBsmtSF', axis = 1, inplace = True)

# GarageCars and GarageArea is basically almost the same thing as it measures size
df_train_filled.drop('GarageCars', axis = 1, inplace = True)
df_test_filled.drop('GarageCars', axis = 1, inplace = True)

In [23]:
y_train = df_train_filled['SalePrice']
df_train_filled.drop('SalePrice' ,axis=1, inplace = True)

In [24]:
all_columns = df_train_filled.columns

In [25]:
cat_columns = ['MSSubClass', 'MSZoning', 'Street',
       'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood',
       'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual',
       'OverallCond', 'YearBuilt', 'YearRemodAdd', 'RoofStyle', 'RoofMatl',
       'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
       'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
       'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 
       'KitchenQual',
        'Functional',  'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive','MoSold', 'YrSold', 'SaleType',
       'SaleCondition']

In [26]:
# Converting all categorical variables to numeric values --> dummy variables afterwards
# Keep track of the groups of each dummy variable set
def solve_cat(train, test):
    test_groups = []
    train_groups = []
    start1 = 0
    start2 = 0
    test_result = pd.DataFrame()
    train_result = pd.DataFrame()
    
    for column in all_columns:
        if(column in cat_columns):
            group1 = pd.get_dummies(train[column])
            group2 = pd.get_dummies(test[column])
            common_cols = list(set(group1.columns).intersection(group2.columns))
            group1 = group1[common_cols]
            group2 = group2[common_cols]
            
            train_groups.append(np.arange(start = start1, stop = start1+len(group1.columns)))
            test_groups.append(np.arange(start = start2, stop = start2+len(group2.columns)))
            start1 += len(group1.columns)
            start2 += len(group2.columns)
            train_result = pd.concat([train_result,group1],axis = 1)
            test_result = pd.concat([test_result,group2],axis = 1)
        else:
            train_groups.append(np.array([start1]))
            test_groups.append(np.array([start2]))
            start1 += 1
            start2 += 1
            train_result = pd.concat([train_result,train[column]],axis = 1)
            test_result = pd.concat([test_result,test[column]],axis = 1)
    
    return train_result, test_result, train_groups, test_groups

In [27]:
train_result, test_result, train_groups, test_groups = solve_cat(df_train_filled, df_test_filled)

In [28]:
train_result

In [29]:
test_result

In [30]:
# Use Sparse Groups Lasso for feature selection (use sparse groups as we have dummy variables)

#cross validation
cv = KFold()
sgl = gpr.SGLCV(
    groups=train_groups, cv=cv, l1_ratio=[0.0, 1.0], tuning_strategy="bayes", n_bayes_iter=50
).fit(train_result, np.log1p(y_train))

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print(
    "train: R^2 = {0:5.3f}, RMSE = {1:7.3f}".format(
        r2_score(np.log1p(y_train), sgl.predict(train_result)),
        np.sqrt(mean_squared_error(np.log1p(y_train), sgl.predict(train_result))),
    )
)

In [32]:
submit = pd.DataFrame({'id': test_result.index.tolist(), 'SalePrice': np.expm1(sgl.predict(test_result))})
submit.to_csv("submission.csv", index=False)

In [33]:
submit